### Part 1: Import Libraries & Data

In [98]:
# Import Libraries
import csv
import pandas as pd 
import numpy as np
import nltk
import os
import nltk.corpus
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Import dataset
data = pd.read_csv('wine.csv')

In [99]:
# Only take the first 5 to test out text mining.
data = data.head(1000)

### Part 2: Tokenize description boxes. 

In [100]:
# Break up the sentences into lists of individual words
for i in range(len(data)):
    text = data['description'][i]
    data['description'][i] = word_tokenize(text.lower())

/Users/amandaAmanda/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [101]:
from nltk.corpus import stopwords
a = set(stopwords.words("english"))

for i in range(len(data)):
    data['description'][i] = [x for x in data['description'][i] if x not in a]

/Users/amandaAmanda/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [102]:
# Part-of-Speech Tagging
# Definitions: https://www.guru99.com/pos-tagging-chunking-nltk.html
# NN is the main part of speech we want to keep here. Everything else can go. 
# - NN = singular

# Add the type of speech to each word in the list. 
for i in range(len(data)):
    words = []
    for token in data['description'][i]:
        words.append(nltk.pos_tag([token]))
        data['description'][i] = words       

/Users/amandaAmanda/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [103]:
for i in range(len(data)):
    new_list = []
    final_set = []
    for tag in range(len(data['description'][i])):
        if data['description'][i][tag][0][1] == 'NN' or data['description'][i][tag][0][1] == 'JJ':
            final_set.append(data['description'][i][tag][0][0])
    data['description'][i] = final_set

/Users/amandaAmanda/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


### Part 3: Match with Flavor Categories

In [104]:
# Create new box, "category"
data['category'] = np.empty((len(data), 0)).tolist()

In [105]:
# Source: https://winefolly.com/deep-dive/wine-tasting-terms-to-use/
# Source 2: https://www.decanter.com/learn/advice/understand-tasting-notes-decoded-344920/#fruit

# ========== Flavors (4) ============ #
# Fruity
fruit_forward_red = ["Raspberry", "Cherry", "Coconut", "Cassis", "Blackberry", "Blueberry", "Jam", "Prune", "Candied", "Raisin", "Spices", "Toffee", "Vanilla", "Tobacco", "Bergamot", "Olive", "Bramble", "Cranberry", "Fig" , "Jammy", "Juniper" , "Kirsch", "Loganberry" , "Plum", "Raisin", "Raspberry" , "Strawberry"]
fruit_forward_white = ["Lemon", "Apple", "Peach", "Mango", "Pear", "Cantaloupe", "Creme Brulee", "Crème Brûlée", "Caramel", "Vanilla", "Apricot", "Banana", "Candied", "Citrus", "Honey", "Gooseberry", "Kiwi", "Lychee", "Marmalade", "Melon", "Orange", "Papaya", "Passion Fruit", "Pineapple", "Prune", "Sherbet"]

# Savory 
savory_red = ["Cranberry", "Soy", "Rhubarb"," Black Currant", "Cassis", "Pepper", "Peppercorn", "Olive", "Strawberry", "Cherry", "Mulberry", "Bilberry", "Wild Blueberry", "Dried Herbs", "Game", "Sage", "Leather", "Tobacco", "Charcoal", "Tar", "Underbrush", "Garrigue", "Gravel", "Torrefaction", "Mineral", "Woodsmoke"]
savory_white = ["Lime", "Lemon", "Pith", "Quince", "Almond", "Apple", "Gooseberry", "Jalapeno", "Grapefruit", "Papaya", "Thyme", "Chervil", "Grass", "Flint", "Chalk", "Chalky", "Petrichor", "Minerally", "Mineral"]

# Earthy
earthy_red = ["Rough", "Tannic", "Rusty", "Rustic", "Earthy", "Balsamic", "Eucalyptus", "Pepper", "Leafy", "Medicinal", "Mint", "Mushroom", "Rhubarb", "Tomato", "beetroot", "tea", "earthy", "meat", "tobacco", "leather", "cardboard", "iodine", "charcoal", "chocolate", "coffee", "leather","tar", "smoke", "wood", "vinyl", "velvet", "velvety"]
earthy_white = ["Asparagus", "Cabbage", "Fennel", "Grass", "Hay", "Hedgerow", "Lemongrass", "Vegetal", "Chalky", "flint", "chalk", "graphite", "mineral", "oyster", "salt", "slate", "steely", "wool", "almond", "beeswax", "biscuit", "brioche", "buttery", "butter", "caramel", "cereal", "cream", "marzipan", "croissant", "pastry", "petrol", "gasoline", "smoky", "Rough", "Tannic", "Rusty", "Rustic", "Earthy", "smoke", "toffee", "vanilla", "walnut", "wax", "match"]
                      
# Floral 
floral_red = ["Lavender", "Peony", "Flower", "Flowery", "Rose", "Turkish Delight", "Violet", "Pepper", "Spice", "Cedar", "Cinnamon", "Clove", "Cola", "Cumin", "Licorice"]
floral_white = ['Camomile', "Geranium", "Elderflower", "Honeysuckle", "Jasmine", "Ginger", "Flower", "Flowery"]
                       
# ========== Body Profile ============ #
# Light-Bodied
light_bodied_red = ["Subtle", "Delicate", "Elegant", "Crisp", "Thin", "Finesse", "Bright", "Floral"]
light_bodied_white = ["Light","Zesty", "Airy", "Lean", "Racy", "Crisp", "Zippy", "Austere", "Long Tingly Finish", "Brilliant", "Lively"]

# Medium-Bodied
med_bodied_red = ["Moderate", "Elegant", "Juicy", "Spicy", "Fleshy", "Tart", "Mellow", "Soft"]

# Full-Bodied
full_bodied_red = ["Rich", "Lush", "Opulent", "Rigid", "Intense", "Bold", "Extracted", "High Alcohol", "High Tannin", "Firm", "Structured", "Muscular", "Concentrated", "Hot"]
full_bodied_white = ["Rich", "Lush", "Oily", "Buttery"]

# ========== Finish ============ #
# Bitter (good when paired with fatty foods)
bitter_red = ["Chewy", "Muscular", "Structured", "Firm", "Rigid", "Closed", "Dried Herbs", "Herby", "Oregano", "Bay Leaf", "Bitter Chocolate", "Baker’s Chocolate", "Bitter Herbs", "Austere", "Angular", "Grippy", "Harsh", "Coarse", "Dense"]
bitter_white = ["Austere", "Citrus Pith", "Quince", "Bitter Almond", "Green Mango", "Green Almond", "Almond", "Chalk"]


In [106]:
# Put in lowercase: 
fruit_forward_red = [x.lower() for x in fruit_forward_red]
fruit_forward_white = [x.lower() for x in fruit_forward_white]

savory_red = [x.lower() for x in savory_red]
savory_white = [x.lower() for x in savory_white]

earthy_red = [x.lower() for x in earthy_red]
earthy_white = [x.lower() for x in earthy_white]

floral_red = [x.lower() for x in floral_red]
floral_white = [x.lower() for x in floral_white]

light_bodied_red = [x.lower() for x in light_bodied_red]
light_bodied_white = [x.lower() for x in light_bodied_white]

med_bodied_red = [x.lower() for x in med_bodied_red]

full_bodied_red = [x.lower() for x in full_bodied_red]
full_bodied_white = [x.lower() for x in full_bodied_white]

bitter_red = [x.lower() for x in bitter_red]
bitter_white = [x.lower() for x in bitter_white]

In [107]:
for i in range(len(data)):
    
    # ======================== Red Wines ================================================================
    if data["type"][i] == ' Red ':
    # =====================================================
    # Fruity Reds
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in fruit_forward_red:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Fruity")
    # =====================================================
    # Savory Reds
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in savory_red:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Savory") 
            
    # =====================================================
    # Earthy Reds
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in earthy_red:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Earthy") 
    # =====================================================
    # Floral Reds
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in floral_red:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Floral") 
    # =====================================================
    # Light-Bodied Red
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in light_bodied_red:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Light-Bodied")
    # =====================================================
    # Medium-Bodied Red
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in med_bodied_red:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Medium-Bodied")
    # =====================================================
    # Full-Bodied Red
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in full_bodied_red:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Full-Bodied")
                     
                
  # ======================== White Wines ================================================================
    elif data["type"][i] == ' White ':
    # =====================================================
    # Fruity Whites
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in fruit_forward_white:
                  sum += 1
        if sum >= 2:
            data["category"][i].append("Fruity")

    # =====================================================
    # Savory Whites
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in savory_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Savory")
    # =====================================================   
    # Earthy Whites
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in earthy_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Earthy") 
    # =====================================================
    # Floral Whites
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in floral_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Floral") 
        
    # =====================================================
    # Light-Bodied White
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in light_bodied_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Light-Bodied")   
    # =====================================================
    # Full-Bodied White
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in full_bodied_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Full-Bodied")  
            
  # ======================== Rosé Wines ================================================================
    else: 
        # Fruity
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in fruit_forward_red or data["description"][i][j] in fruit_forward_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Fruity")
            
        # Savory 
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in savory_red or data["description"][i][j] in savory_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Savory") 
            
       # Earthy 
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in earthy_red or data["description"][i][j] in earthy_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Savory") 
            
            
        # Floral
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in floral_red or data["description"][i][j] in floral_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Savory") 
            
            
        # Light-Bodied
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in light_bodied_red or data["description"][i][j] in light_bodied_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Light-Bodied")  
                 
        # Full-Bodied 
        sum = 0
        for j in range(len(data["description"][i])):
            if data["description"][i][j] in full_bodied_red or data["description"][i][j] in full_bodied_white:
                sum += 1
        if sum >= 2:
            data["category"][i].append("Full-Bodied")

In [108]:
data

,ything,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,type,category
0,1.0,Portugal,"[ripe, fruity, wine, smooth, firm, juicy, red,...",Avidagos,87.0,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red,Portuguese Red,Quinta dos Avidagos,Red,[]
1,2.0,US,"[tart, snappy, lime, flesh, rind, dominate, gr...",NaN,87.0,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris,Pinot Gris,Rainstorm,White,[]
2,3.0,US,"[pineapple, rind, lemon, pith, orange, blossom...",Reserve Late Harvest,87.0,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling,Riesling,St. Julian,White,"[Fruity, Savory]"
3,4.0,US,"[much, regular, rough, tannic, rustic, earthy,...",Vintner's Reserve Wild Child Block,87.0,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Red,[Earthy]
4,5.0,Spain,"[blackberry, raspberry, show, typical, navarra...",Ars In Vitro,87.0,15.0,Northern Spain,Navarra,NaN,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot,Tempranillo-Merlot,Tandem,White,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1299.0,Italy,"[suggest, red, berry, plum, hint, cake, spice,...",Cornell Villa Nigra,89.0,49.0,Northeastern Italy,Alto Adige,NaN,Kerin O’Keefe,@kerinokeefe,Colterenzio 2011 Cornell Villa Nigra Pinot Nero,Pinot Nero,Colterenzio,Red,"[Fruity, Savory]"
996,1300.0,US,"[successful, straight-ahead, chardonnay, butte...",NaN,89.0,35.0,California,Santa Barbara County,Central Coast,Matt Kettmann,@mattkettmann,Demetria 2012 Chardonnay,Chardonnay,Demetria,White,"[Fruity, Earthy]"
997,1301.0,France,"[herbaceous, ripe, wine, grapefruit, lemon, ze...",NaN,89.0,20.0,Loire Valley,Menetou-Salon,NaN,Roger Voss,@vossroger,Domaine de l'Ermitage 2013 Menetou-Salon,Sauvignon Blanc,Domaine de l'Ermitage,White,"[Fruity, Savory]"
998,1302.0,France,"[ripe, fresh, fruity, wine, tangy, mineral, te...",NaN,89.0,21.0,Loire Valley,Sancerre,NaN,Roger Voss,@vossroger,Domaine de Rome 2013 Sancerre,Sauvignon Blanc,Domaine de Rome,White,[Savory]


In [97]:
# Save back to CSV
data.to_csv("wine_modified.csv", index=False)